## Required installs/imports


In [ ]:
# Installs the library used to utilise modern PRNGs
!pip install pure-prng

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 21.0 MB/s eta 0:00:00


In [ ]:
# Used to round floats
import numpy as np
# Aids seeding by allowing the current time to be used as a seed
import time
# Provides python implementations of common PRNGs
from pure_prng_package import pure_prng

# Necessary imports to allow model development
import numpy as np
import tensorflow as tf
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam

## Classes to provide data generation/evaluation methods

In [ ]:
# Class used to access my implmentations of PRNGs
class TestManagement():
  # Initialises object and sets the default seed
  def __init__(self):
      self.seed = 0

  # Method to seed the PRNG (seeds all PRNGs in class)
  def seed_PRNG(self, seed):
    self.seed = seed

  def bit_success(self, model, inputData, trueOutputs, sequence_length):
    """ 
    Method to evaluate the provided model and store the amount of
    successful predictions for each bit of the output
    :param model: keras model - Model used to generate predicitions
    :param trueOutputs: list[list[int]]- List containing the expected y outputs
    :param sequence_length: int - Length of generated binary string being predicited
    :return list[int] - Amount of successful predictions for each bit
    """
    # Set initial amount of successful predictions for each bit to zero
    successfulPredicts = [0]*sequence_length
    # Feeds the input data to the model and stores the predictions made
    predicted = (model.predict(inputData).round())
    # Iterate over all outputed data
    for testIndex in range(0, len(inputData)):
      # Iterate over each bit in output
      for i in range(sequence_length):
        # If the predicted bit matches the true bit value then increment the successful predicts for the current bit
        if predicted[testIndex][i] == trueOutputs[testIndex][i]: successfulPredicts[i] += 1
        # Prediction may be greater than 1 if the prediction is made with high certainity
        elif predicted[testIndex][i] > 1 and trueOutputs[testIndex][i] == 1: self.successfulPredicts[i] += 1

    return successfulPredicts
   

  def zero_only_PRNG(self, length=100):
    """ 
    Returns a binary string containing only 0 of specified length.
    Used to test for major flaws in models
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    return "0" * length        


  def alternating_bits_PRNG(self, length=100):
    """ 
    Returns output of a basic PRNG implementation that alernates each bit (010101)
    :param length: int - Length of generated binary string
    :return string - generated binary string
    """
    # Use seed to determine the starting bit of the generated binary string
    self.seed = self.seed%2
    # Utilises efficent method to repeat a string pattern
    if (self.seed == 1):
        output = "10" * int(length/2)
    else:
        output = "01" * int(length/2)

    # Length of generated binary string is odd
    if (length%2 == 1):
      # Add final bit to string
      output += str(self.seed)
      # Set the new seed value
      if (self.seed == 0): self.seed = 1
      else: self.seed = 0
    
    return output


  def alternating_num_PRNG(self):
    """ 
    Returns output of a basic PRNG implementation that alernates between two binary strings
    :return string - generated binary string
    """
    # Use seed to determine the binary string to be returned
    self.seed = (self.seed+1)%2
    if (self.seed == 0):
      # Convert integer to a binary string 
      randomBinary = str(bin(1643712566))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary
    else:
      # Convert integer to a binary string 
      randomBinary = str(bin(2372817037))[2:]
      # Returns binary string after ensuring a minimum length of 32
      return (32-len(randomBinary))*"0" + randomBinary


  def basic_equation_based(self):
    """ 
    Returns output of a very weak equation based PRNG implementation
    Expected to be predicted near perfectly
    :return string - generated binary string
    """
    # Generates random number using seed
    self.seed = (20 * self.seed + 52) % 2**32
    # Converts generated number to a binary string
    randomBinary = str(bin(self.seed))[2:]
    # Returns binary string after ensuring a minimum length of 32
    return (32-len(randomBinary))*"0" + randomBinary


  def basic_equation_based2(self):
      self.seed = (36791 * self.seed + 83247) % 2**32
      randomBinary = str(bin(self.seed))[2:]
      return (32-len(randomBinary))*"0" + randomBinary


# Expects odd starting seed
  def poor_equation_based(self):
    self.seed = (65539 * self.seed + 0) % 2**31
    randomBinary = str(bin(self.seed))[2:]
    return (32-len(randomBinary))*"0" + randomBinary

TestHandler = TestManagement()

In [ ]:
class QCGManagement():
  def __init__(self):
    self.currentGen = pure_prng(int(time.time()), prng_type="QCG").source_random_number()

  def seed_current(self, seed=time.time()):
    self.currentGen = pure_prng(int(seed), prng_type="QCG").source_random_number()
  
  def output_current(self):
    return str(bin(next(self.currentGen)))[2:]

QCGHandler = QCGManagement()

In [ ]:
class Ran64Management():
  def __init__(self):
    self.currentGen = pure_prng(int(time.time()), prng_type="Ran64").source_random_number()

  def seed_current(self, seed=time.time()):
    self.currentGen = pure_prng(int(seed), prng_type="Ran64").source_random_number()
  
  def output_current(self):
    return str(bin(next(self.currentGen)))[2:]

Ran64Handler = Ran64Management()

## Data generation function to be used during data production

In [ ]:
def seed_current(seed): Ran64Handler.seed_current(seed)

def output_current(): 
  randomBinary = Ran64Handler.output_current()
  return (64-len(randomBinary))*"0" + randomBinary

## Setting parameters for data/model

In [ ]:
### Setting parameters
# Sets paramemters for generating train/test data
num_samples = 1000
sequence_length = 64
samplesConcatenated = 15
# Sets parameters for produced model
epochs = 10
batch_size = 100

## Produces data for training/testing

In [ ]:
### Generation of data for training/testing
# Seeding
print("Seeding generator")
seed_current(55)

 
print("Generating data")
X_data = []
Y_data = []
# Create the amount of samples specified
for i in range(0, num_samples):
  sampleData = []
  # Concatenate amount of samples specified for x data
  for j in range(0, samplesConcatenated):
    sampleData += [int(bit) for bit in output_current()]
  X_data.append(sampleData)
  Y_data.append([int(bit) for bit in output_current()])

Seeding generator
Generating data


## Splitting data into training/testing sets

In [ ]:
### Separate data into training/testing sets
# Percentage of data used for testing the created prediction model
testDataPerc = 0.2
dataSplit = int(num_samples*(1-testDataPerc))
# Separates x data
x_train = X_data[:dataSplit]
x_test = X_data[dataSplit:]
del X_data
# Separates y data
y_train = Y_data[:dataSplit]
y_test = Y_data[dataSplit:]
del Y_data
#x_train, x_test, Y_data, y_test = train_test_split(X_data, Y_data, test_size = testDataPerc, random_state = 5)


## Creation of model

In [ ]:
#optimizer = Adam(learning_rate=0.05)
strategy = tf.distribute.OneDeviceStrategy('/gpu:0')

print("Creating model")
# Model compilation
with strategy.scope():
  model = Sequential()
  model.add(LSTM(sequence_length, input_shape=(sequence_length*samplesConcatenated, 1), return_sequences=True))
  model.add(LSTM(int(sequence_length//1.5), return_sequences=True))
  model.add(LSTM(sequence_length//2))
  model.add(Dense(sequence_length, activation='relu'))

  model.compile(loss='binary_crossentropy', optimizer="Adam", metrics=['accuracy'])

Creating model


## Training model

In [ ]:
# Train the model with the x/y train data and validate using the test data after each epoch
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=batch_size)

Epoch 1/10
12/12 [==============================] - 40s 3s/step - loss: 4.0746 - accuracy: 0.0000e+00 - val_loss: 3.4706 - val_accuracy: 0.0000e+00
Epoch 2/10
12/12 [==============================] - 29s 2s/step - loss: 3.1764 - accuracy: 0.0000e+00 - val_loss: 2.9586 - val_accuracy: 0.0000e+00
Epoch 3/10
12/12 [==============================] - 28s 2s/step - loss: 2.8931 - accuracy: 0.0025 - val_loss: 2.7153 - val_accuracy: 0.0000e+00
Epoch 4/10
12/12 [==============================] - 28s 2s/step - loss: 2.6478 - accuracy: 0.0000e+00 - val_loss: 2.5792 - val_accuracy: 0.0000e+00
Epoch 5/10
12/12 [==============================] - 32s 3s/step - loss: 2.5712 - accuracy: 0.0000e+00 - val_loss: 2.5626 - val_accuracy: 0.0000e+00
Epoch 6/10
12/12 [==============================] - 28s 2s/step - loss: 2.5578 - accuracy: 0.0000e+00 - val_loss: 2.5570 - val_accuracy: 0.0000e+00
Epoch 7/10
12/12 [==============================] - 31s 3s/step - loss: 2.5536 - accuracy: 0.0000e+00 - val_loss: 2.

In [ ]:
# Saves the model as a file
model.save("Ran64FullPredict1.h5")

## Evaluating predictability of each bit

In [ ]:
### Produces data for detailed evaluation
# Amount of samples used for evaluation
eval_samples = 5000


x_test = []
y_test = []
# Create the amount of samples specified
for i in range(0, eval_samples):
  sampleData = []
  # 'samplesConcatenated' was defined when creating the data for model training
  # Concatenate amount of samples specified for x data
  for j in range(0, samplesConcatenated):
    sampleData += [int(bit) for bit in output_current()]
  x_test.append(sampleData)
  y_test.append([int(bit) for bit in output_current()])


In [ ]:
# Feeds data to model and determines the amount of successful predictions for each bit index
successfulPredictions=TestHandler.bit_success(model, x_test, y_test, sequence_length)
# Prints information regarding the most predictable bit
print("Highest predict rate: {rate:.2f}%\nIndex: {index}".format(rate=100*max(successfulPredictions)/eval_samples, index=successfulPredictions.index(max(successfulPredictions))))

Highest predict rate: 51.74%
Index: 26


In [ ]:
# Creates a list contaiing each bit index and the successful predictions for the bit
indexSuccesses = []
for i in range(len(successfulPredictions)):
  indexSuccesses.append([successfulPredictions[i], i])
# Sorts the list in descending order 
indexSuccesses.sort(reverse=True)

# Outputs the successful prediction rate for each bit
for i in range(len(indexSuccesses)):
  print("Predict rate: {rate:.2f}%   |   Index: {index}".format(rate=100*(indexSuccesses[i][0]/eval_samples), index=indexSuccesses[i][1]))

Predict rate: 51.74%   |   Index: 26
Predict rate: 51.60%   |   Index: 32
Predict rate: 51.28%   |   Index: 60
Predict rate: 51.26%   |   Index: 13
Predict rate: 51.22%   |   Index: 14
Predict rate: 51.20%   |   Index: 12
Predict rate: 51.18%   |   Index: 54
Predict rate: 51.00%   |   Index: 62
Predict rate: 50.92%   |   Index: 38
Predict rate: 50.84%   |   Index: 28
Predict rate: 50.60%   |   Index: 59
Predict rate: 50.60%   |   Index: 53
Predict rate: 50.60%   |   Index: 24
Predict rate: 50.60%   |   Index: 5
Predict rate: 50.48%   |   Index: 22
Predict rate: 50.44%   |   Index: 29
Predict rate: 50.34%   |   Index: 15
Predict rate: 50.32%   |   Index: 39
Predict rate: 50.30%   |   Index: 55
Predict rate: 50.28%   |   Index: 45
Predict rate: 50.28%   |   Index: 21
Predict rate: 50.24%   |   Index: 4
Predict rate: 50.18%   |   Index: 40
Predict rate: 50.14%   |   Index: 37
Predict rate: 50.14%   |   Index: 33
Predict rate: 50.12%   |   Index: 0
Predict rate: 50.10%   |   Index: 25
Pred